In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf

spark = SparkSession. \
builder. \
appName('UNIR'). \
config("spark.executor.memory", "1g"). \
config("spark.driver.memory", "2g"). \
config('spark.dirver.maxResultSize', '1g'). \
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0"). \
getOrCreate()

In [ ]:
#Abrir terminal vagrant ssh
#Ejecutar en la terminal de vagrant : 
kafka-topics.sh --bootstrap-server localhost:9092 --create --topic promedios --partitions 1 --replication-factor 1

kafka-topics.sh --bootstrap-server localhost:9092 --list

kafka-console-producer.sh --topic promedios --bootstrap-server localhost:9092

In [ ]:
def promediarValores(df):
   df.createOrReplaceTempView("resultadoMedio")
   promedios = spark.sql("""SELECT tipo, AVG(total) AS promedio FROM resultadoMedio GROUP BY tipo ORDER BY promedio DESC""")
   return promedios

In [ ]:
tiposStreamingDF = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "127.0.0.1:9092")
  .option("subscribe", "promedios")
  .load())

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pyspark.sql.functions as F
esquema = StructType([\
  StructField("tipo", StringType()),\
  StructField("total", DoubleType())\
])

In [ ]:
parsedDF = tiposStreamingDF\
     .select("value")\
     .withColumn("value", F.col("value").cast(StringType()))\
     .withColumn("parejas", F.from_json(F.col("value"), esquema))\
     .withColumn("tipo", F.col("parejas.tipo"))\
     .withColumn("total", F.col("parejas.total"))

In [ ]:
promediosStreamingDF = promediarValores(parsedDF)

In [ ]:
salida = promediosStreamingDF\
                    .writeStream\
                    .queryName("AgregacionPromedios")\
                    .outputMode("complete")\
                    .format("memory")\
                    .option("checkpointLocation", "hdfs://localhost:9000/tmp/checkpoints/promedios")\
                    .start()

In [ ]:
promediosDF = spark.sql("select * from AgregacionPromedios")

In [ ]:
promediosDF.show()

In [ ]:
{"tipo": "gasto", "total": 3.5}
{"tipo": "ingreso", "total": 7.0}
{"tipo": "ingreso", "total": 6.5}
{"tipo": "gasto", "total": 1.5}
{"tipo": "gasto", "total": 2.5}

In [ ]:
promediosDF.show()

In [ ]:
promediosDF.show()

In [ ]:
promediosDF.show()